# Anomaly detection over heterogeneous data streams

The recipe, inspired from [this paper](https://arxiv.org/abs/1812.02848) by Palladino and Thissen:

1. Split time line into intervals; split the data streams into subsets corresponding to these intervals.
1. For each interval:
    1. Compose a graph out of the events belonging to the interval.
    1. Extract a vector topological features for each vertex.
    1. Reduce the dimension of these feature vectors.
    1. Compute the anomaly indicator according to the reduced feature vectors for the interval.
    
These various steps will be detailed as we go along.

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120 --ignore W293,E302
%pdb on

Automatic pdb calling has been turned ON


In [2]:
import notebooks_as_modules

In [3]:
from jupytest import Suite, Report, Magic, fail, summarize_results, assert_, eq, Explanation, ExplanationOnFailure, join_args

In [4]:
suite = Suite('test')
if __name__ == '__main__':
    suite |= Report()
    suite |= Magic()

In [5]:
from growing import growing

## Time line partition

Each data stream is taken in as a Dask dataframe, itself indexed by a certain key. A heterogeneous data stream is a set of named data streams, all indexed in the same key.

In [6]:
import dask.dataframe as ddf
from typing import Mapping, TypeVar

DomainIndex = TypeVar("DomainIndex")
Schema = Mapping[str, str]
StreamHeterogeneous = Mapping[str, ddf.DataFrame]  # Data frame is indexed by DomainIndex

Given a subset of index values (articulated as an ordered, strictly monotonic increasing iterable sequence), a *partition* is thus a heterogeneous stream, mapped by these values.

In [7]:
import pandas as pd
from typing import Mapping, Tuple

Partition = Tuple[DomainIndex, StreamHeterogeneous]
Partitioning = Mapping[DomainIndex, StreamHeterogeneous]

### Partitioning sequence

The partitioning sequence is processed into a sequence of slices, with which to interrogate the index of the heterogeneous stream. Namely, a sequence `[i0, i1, i2... iN]` yields mutually exclusive slices `i0:(i1-1)`, `i1:(i2-1)`... `iN-1:((iN)-1)`. Let's embody partition sequences into a class.

In [8]:
from abc import ABC, abstractmethod
from typing import Generic, Sequence, Iterator, Tuple


class SequencePartition(ABC, Generic[DomainIndex]):

    @abstractmethod
    def __iter__(self) -> Iterator[DomainIndex]:
        raise NotImplementedError()
        yield 0
        
    @abstractmethod
    def pred(self, n: DomainIndex) -> DomainIndex:
        raise NotImplementedError()
        
    def slices(self) -> Iterator[Tuple[DomainIndex, DomainIndex]]:
        prev: Optional[DomainIndex] = None
        for cur in self:
            if prev is not None:
                yield (prev, self.pred(cur))
            prev = cur

In [9]:
class SequenceInt(SequencePartition[int]):
    
    def __init__(self, seq: Sequence[int]) -> None:
        super().__init__()
        self._seq = seq
        
    def __iter__(self) -> Iterator[int]:
        return iter(self._seq)
    
    def pred(self, n: int) -> int:
        return n - 1

In [10]:
%%test No slice for a sequence of less than two items
for i, seq in enumerate([[], [0]]):
    assert_(eq, actual=len(list(SequenceInt(seq).slices())), expected=0)

Test No slice for a sequence of less than two items passed.


In [11]:
%%test Iterating single slice
assert_(eq, expected=[(0, 1)], actual=list(SequenceInt([0, 2]).slices()))

Test Iterating single slice passed.


In [12]:
%%test With sequences not strictly monotonic, we get degenerate slices
assert_(eq, expected=[(0, 0), (1, 0)], actual=list(SequenceInt([0, 1, 1]).slices()))

Test With sequences not strictly monotonic, we get degenerate slices passed.


In [13]:
%%test Multiple slices from a sequence of integers
assert_(eq, expected=[(0, 2), (3, 4)], actual=list(SequenceInt([0, 3, 5]).slices()))

Test Multiple slices from a sequence of integers passed.


### Partitioning sequence of date-times

In [14]:
from typing import Any


class SequenceDatetime(SequencePartition[pd.Timestamp]):
    
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__()
        self._seq = pd.date_range(*args, **kwargs)
        
    def __iter__(self) -> Iterator[pd.Timestamp]:
        return iter(self._seq)
        
    def pred(self, ts: pd.Timestamp) -> pd.Timestamp:
        return ts - pd.Timedelta(nanoseconds=1)

In [15]:
%%test Multiple slices from a date-time partitioning sequence
assert_(
    eq,
    expected=[
        (pd.Timestamp("2015-01-01T00:00:00"), pd.Timestamp("2015-01-01T00:59:59.999999999")),
        (pd.Timestamp("2015-01-01T01:00:00"), pd.Timestamp("2015-01-01T01:59:59.999999999")),
        (pd.Timestamp("2015-01-01T02:00:00"), pd.Timestamp("2015-01-01T02:59:59.999999999")),
        (pd.Timestamp("2015-01-01T03:00:00"), pd.Timestamp("2015-01-01T03:59:59.999999999"))
    ],
    actual=list(SequenceDatetime("2015-01-01T00:00:00", "2015-01-01T04:00:00", freq=pd.Timedelta(hours=1)).slices())
)

Test Multiple slices from a date-time partitioning sequence passed.


### Partitioning of the heterogeneous stream

In [16]:
def partition(sh: StreamHeterogeneous, seq: SequencePartition) -> Partitioning:
    P = {}
    for lower, upper in seq.slices():
        P[lower] = {name: df.loc[lower:upper] for name, df in sh.items()}
    return P

In [17]:
import dask.dataframe as ddf
from jupytest import Explanation
import pandas as pd


def equals(expected: pd.DataFrame, actual: ddf.DataFrame) -> Explanation:
    if actual.compute().equals(expected):
        return True
    return Explanation(
        "These dataframes are not equal as expected",
        [("Expected", expected), ("Actual", actual)]
    )

In [18]:
%%test Partitioning an int-indexed heterogeneous stream
import dask


@dask.delayed
def f1():
    return pd.DataFrame({"a": [3, 8, 9], "b": [10, 102, 89]}, index=pd.Int64Index([1, 4, 6]))

@dask.delayed
def f2():
    return pd.DataFrame({"a": [5, 3], "b": [90, 32]}, index=pd.Int64Index([8, 11]))

@dask.delayed
def f3():
    return pd.DataFrame({"a": [2, 11, 7, 3], "b": [56, 98, 99, 67]}, index=pd.Int64Index([13, 14, 15, 19]))

@dask.delayed
def g1():
    return pd.DataFrame(
        {"a": [6, 3, 8, -1, 0], "c": [-3, -8, -11, -7, -10], "d": [106, 203, 267, 308, 429]},
        index=pd.Int64Index([3, 6, 7, 10, 11])
    )

@dask.delayed
def g2():
    return pd.DataFrame(
        {"a": [2, 8, 9, 5, 5, 11], "c": [-13, -6, -7, -6, -8, -2], "d": [567, 308, 289, 290, 432, 387]},
        index=pd.Int64Index([14, 15, 17, 19, 21, 24])
    )


sh = {
    "p": ddf.from_delayed(
        [f1(), f2(), f3()],
        meta={"a": "int64", "b": "int64"},
        divisions=[1, 8, 13, 20]
    ),
    "q": ddf.from_delayed(
        [g1(), g2()],
        meta={"a": "int64", "c": "int64", "d": "int64"}, 
        divisions=[3, 14, 28]
    )
}
P = partition(sh, SequenceInt([0, 5, 10, 15, 20]))

assert_(eq, actual=set(P.keys()), expected={0, 5, 10, 15})
assert_(eq, reference={"p", "q"}, **{str(i): set(p.keys()) for i, p in enumerate(P.values())})

assert_(equals, actual=P[0]["p"], expected=pd.DataFrame({"a": [3, 8], "b": [10, 102]}, index=[1, 4]))
assert_(equals, actual=P[0]["q"], expected=pd.DataFrame({"a": [6], "c": [-3], "d": [106]}, index=[3]))
assert_(equals, actual=P[5]["p"], expected=pd.DataFrame({"a": [9, 5], "b": [89, 90]}, index=[6, 8]))
assert_(
    equals,
    actual=P[5]["q"],
    expected=pd.DataFrame({"a": [3, 8], "c": [-8, -11], "d": [203, 267]}, index=[6, 7])
)
assert_(
    equals,
    actual=P[10]["p"],
    expected=pd.DataFrame({"a": [3, 2, 11], "b": [32, 56, 98]}, index=[11, 13, 14])
)
assert_(
    equals,
    actual=P[10]["q"],
    expected=pd.DataFrame(
        {"a": [-1, 0, 2], "c": [-7, -10, -13], "d": [308, 429, 567]},
        index=[10, 11, 14]
    )
)
assert_(
    equals,
    actual=P[15]["p"],
    expected=pd.DataFrame({"a": [7, 3], "b": [99, 67]}, index=[15, 19])
)
assert_(
    equals,
    actual=P[15]["q"],
    expected=pd.DataFrame(
        {"a": [8, 9, 5], "c": [-6, -7, -6], "d": [308, 289, 290]},
        index=[15, 17, 19]
    )
)

Test Partitioning an int-indexed heterogeneous stream passed.


## Crunching event sets into artifact graphs

Step 2A above creating an *artifact graph* for each data partition. These graphs involve the decomposition of each event of a partition into a set of *artifacts*, entities associated to typical IT concepts, often acting as signifiers for a host, a computation, a network. The artifact graph is composed as follows:

1. Each artifact is associated to a vertex in the graph.
1. The artifacts extracted from an event E form a clique within the graph.
1. When adding an event clique to the graph, if an edge already exist, we increment its weight.

### Graph architecture

Determining (and engineering) which elements of an event are artifacts is a matter of arbitrary policy. We will encode this policy into a *graph architecture* object.

In [19]:
from abc import ABC, abstractmethod
from typing import Mapping, Sequence, NewType, NamedTuple


Artifact = Tuple[str, str]
Schema = Mapping[str, str]


class ArchGraph(ABC):
    
    @abstractmethod
    def get_artifacts(self, event: NamedTuple, schema: Schema) -> Sequence[Artifact]:
        """
        Extracts the artifacts from an event, itself encoded as a ad hoc namedtuple (hence the absence)
        of typing information.
        """
        raise NotImplementedError()

The typical architecture is to use all event attributes of dtype `object` as artifacts.

In [20]:
class AllObjects(ArchGraph):
    
    def get_artifacts(self, event: NamedTuple, dtypes: Schema) -> Sequence[Artifact]:
        artifacts = []
        for attr, dtype in dtypes.items():
            if dtype == "object":
                try:
                    artifacts.append((attr, getattr(event, attr)))
                except AttributeError:
                    # We ignore the absence of any attribute.
                    pass
        return artifacts

An event factory for testing:

In [21]:
from collections import namedtuple
from contextlib import contextmanager
from typing import ContextManager


@contextmanager
def test_event_factory(**fields: Any) -> ContextManager[namedtuple]:
    type_event = namedtuple("TestEvent", fields.keys())
    yield type_event(**fields)

In [22]:
%%test No artifact provided when the dtype catalog is empty
with test_event_factory(a="asdf", b="qwer", c=38) as event:
    assert AllObjects().get_artifacts(event, {}) == []

Test No artifact provided when the dtype catalog is empty passed.


In [23]:
%%test Event with no attribute yields no artifact
with test_event_factory() as event:
    assert AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"}) == []

Test Event with no attribute yields no artifact passed.


In [24]:
%%test AllObjects yields all object attributes as artifacts
with test_event_factory(a="asdf", b="qwer", c=38) as event:
    assert sorted(AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"})) == \
        [("a", "asdf"), ("b", "qwer")]

Test AllObjects yields all object attributes as artifacts passed.


In [25]:
%%test Missing artifact compared to the dtype catalog is silently ignored
with test_event_factory(a="asdf", c=38) as event:
    assert AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"}) == [("a", "asdf")]

Test Missing artifact compared to the dtype catalog is silently ignored passed.


In [26]:
%%test Extraneous attributes compared to the dtype catalog are ignored
with test_event_factory(a="asdf", b="qwer", d="zxcv") as event:
    assert sorted(AllObjects().get_artifacts(event, {"a": "object", "b": "object", "c": "int64"})) == \
        [("a", "asdf"), ("b", "qwer")]

Test Extraneous attributes compared to the dtype catalog are ignored passed.


### Adding artifact cliques to a graph

In [27]:
import igraph as ig
from itertools import combinations


def add_clique(g: ig.Graph, clique: Sequence[Artifact]) -> ig.Graph:
    dummy = g.add_vertex("__DUMMY__")  # Necessary to query the main vertex sequence.

    vertices = []
    for type_artifact, value_artifact in clique:
        name = f"{type_artifact}:{value_artifact}"
        vertices += (g.vs(name=name) or [g.add_vertex(name)])

    for src, dst in combinations(vertices, 2):
        for edge in (g.es(_from=src, _to=dst) or [g.add_edge(src, dst, weight=0)]):
            edge["weight"] += 1
            
    g.delete_vertices(dummy)
    return g

In [28]:
import igraph as ig


def test_graph() -> ig.Graph:
    g = ig.Graph()
    v_asdf = g.add_vertex("a:asdf")
    v_qwer = g.add_vertex("b:qwer")
    g.add_edge(v_asdf, v_qwer, weight=1)
    return g

#### Comparing graphs by reducing them to tuples and sets

In [29]:
from typing import List


ListVertices = List[str]
ListEdgesWeighted = List[Tuple[str, str, int]]
GraphLists = Tuple[ListVertices, ListEdgesWeighted]


def graph2sets(g: ig.Graph) -> GraphLists:
    return (
        sorted([v["name"] for v in g.vs]),
        sorted([tuple(sorted([v["name"] for v in e.vertex_tuple])) + (e["weight"],) for e in g.es])
    )

In [30]:
%%test Graph sets for simple graph
assert_(eq, actual=graph2sets(test_graph()), expected=(["a:asdf", "b:qwer"], [("a:asdf", "b:qwer", 1)]))

Test Graph sets for simple graph passed.


In [31]:
%%test Graph sets for bit more complex graph
g = ig.Graph()
for name in ["a", "b", "c", "1", "2", "3"]:
    g.add_vertex(name)
g.add_edge("a", "b", weight=1)
g.add_edge("c", "b", weight=3)
g.add_edge("1", "2", weight=5)
g.add_edge("3", "2", weight=0)
g.add_edge("b", "2", weight=10)

assert_(
    eq,
    expected=(
        ["1", "2", "3", "a", "b", "c"],
        [("1", "2", 5), ("2", "3", 0), ("2", "b", 10), ("a", "b", 1), ("b", "c", 3)]
    ),
    actual=graph2sets(g)
)

Test Graph sets for bit more complex graph passed.


In [32]:
%%test Graph sets don't care about vertex addition order
g1 = ig.Graph()
g1.add_vertex("a")
g1.add_vertex("b")
g1.add_edge("a", "b", weight=1)

g2 = ig.Graph()
g2.add_vertex("b")
g2.add_vertex("a")
g2.add_edge("b", "a", weight=1)

assert_(eq, ordered=graph2sets(g1), unordered=graph2sets(g2))

Test Graph sets don't care about vertex addition order passed.


In [33]:
%%test Graph sets when no edge
g = ig.Graph()
for name in ["a", "b"]:
    g.add_vertex(name)
    
assert_(eq, actual=graph2sets(g), expected=(["a", "b"], []))

Test Graph sets when no edge passed.


#### Unit tests for clique adding

In [34]:
import igraph as ig

def are_graphs_equal(**graphs: ig.Graph) -> ExplanationOnFailure:
    sets = {name: graph2sets(g) for name, g in graphs.items()}
    if eq(**sets):
        return True
    return Explanation("These graphs are not equal", join_args([], sets))

In [35]:
%%test Adding a clique without any artifact leaves the graph unchanged
expected = test_graph()
actual = add_clique(test_graph(), [])
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique without any artifact leaves the graph unchanged passed.


In [36]:
%%test Add a clique to an empty graph
expected = test_graph()
actual = ig.Graph()
assert_(are_graphs_equal, expected=expected, actual=add_clique(ig.Graph(), [("a", "asdf"), ("b", "qwer")]))

Test Add a clique to an empty graph passed.


In [37]:
%%test Adding a clique of all new vertices
clique: Sequence[Artifact] = [("c", "uiop"), ("d", "qwer"), ("e", "zxcv"), ("f", "hjkl")]

expected = test_graph()
for t, v in clique:
    expected.add_vertex(f"{t}:{v}")
for s, e in [
    ("c:uiop", "d:qwer"), ("c:uiop", "e:zxcv"), ("c:uiop", "f:hjkl"),
    ("d:qwer", "e:zxcv"), ("d:qwer", "f:hjkl"),
    ("e:zxcv", "f:hjkl")
]:
    expected.add_edge(s, e, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique of all new vertices passed.


In [38]:
%%test Adding a clique that intersects another one fully
clique: Sequence[Artifact] = [("a", "asdf"), ("b", "qwer"), ("c", "zxcv")]

expected = test_graph()
a = next(iter(expected.vs(name="a:asdf")))
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
for e in expected.es(_from=a, _to=b):
    e["weight"] = 2
expected.add_edge(a, c, weight=1)
expected.add_edge(b, c, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another one fully passed.


In [39]:
%%test Adding a clique that intersects another one by one vertex only
clique = [("b", "qwer"), ("c", "zxcv")]

expected = test_graph()
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
expected.add_edge(b, c, weight=1)

actual = add_clique(test_graph(), clique)
assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another one by one vertex only passed.


In [40]:
%%test Adding a clique that intersects another partly, but by more than a single vertex
clique = [("a", "asdf"), ("c", "zxcv"), ("d", "hjkl")]

expected = test_graph()
a = next(iter(expected.vs(name="a:asdf")))
b = next(iter(expected.vs(name="b:qwer")))
c = expected.add_vertex("c:zxcv")
expected.add_edge(a, c, weight=2)
expected.add_edge(b, c, weight=1)
d = expected.add_vertex("d:hjkl")
expected.add_edge(a, d, weight=1)
expected.add_edge(c, d, weight=1)

actual = test_graph()
actual.add_vertex("c:zxcv")
actual.add_edge(a, c, weight=1)
actual.add_edge(b, c, weight=1)
add_clique(actual, clique)

assert_(are_graphs_equal, expected=expected, actual=actual)

Test Adding a clique that intersects another partly, but by more than a single vertex passed.


### Building the graph

In [41]:
import igraph as ig
from memo import memo


@memo
def build_graph_artifacts(sh: StreamHeterogeneous, arch: ArchGraph) -> ig.Graph:
    graph = ig.Graph()
    for name_stream, df_stream in sh.items():
        schema: Schema = {n: str(t) for n, t in df_stream.dtypes.items()}
        for event in df_stream.itertuples():
            add_clique(graph, arch.get_artifacts(event, schema))
    return graph

In [42]:
%%test Building an artifact graph
from memo import suspending_memoization

sh = {
    "asdf": pd.DataFrame(
        data={
            "X": ["a.b.a.b", "a.c.c.b", "a.b.a.c", "a.a.a.a", "b.a.b.c"],
            "Y": ["443", "21", "22", "443", "443"]
        },
        index=[1, 3, 7, 8, 9]
    ),
    "qwer": pd.DataFrame(
        data={
            "W": ["10.2.3.1", "10.2.128.3", "10.2.1.1", "10.10.1.1", "10.2.128.38", "10.10.1.43"],
            "X": ["a.b.a.b", "a.e.a.d", "a.a.b.b", "a.b.a.c", "b.c.b.c", "a.a.a.a"],
            "Z": ["asdf.exe", "qwer.exe", "zxcv.exe", "qwer.exe", "./hkjl", "asdf.exe"]
        },
        index=[0, 3, 4, 6, 8, 9]
    )
}

expected = ig.Graph()
for v in [
    "X:a.b.a.b", "X:a.c.c.b", "X:a.b.a.c", "X:a.a.a.a", "X:b.a.b.c", "X:a.e.a.d", "X:a.a.b.b", "X:b.c.b.c",
    "Y:443", "Y:21", "Y:22",
    "W:10.2.3.1", "W:10.2.128.3", "W:10.2.1.1", "W:10.10.1.1", "W:10.2.128.38", "W:10.10.1.43",
    "Z:asdf.exe", "Z:qwer.exe", "Z:zxcv.exe", "Z:./hkjl"
]:
    expected.add_vertex(v)
for s, e in zip(
    ["X:a.b.a.b", "X:a.c.c.b", "X:a.b.a.c", "X:a.a.a.a", "X:b.a.b.c"],
    ["Y:443", "Y:21", "Y:22", "Y:443", "Y:443"]
):
    expected.add_edge(s, e, weight=1)
for a, b, c in zip(
    ["W:10.2.3.1", "W:10.2.128.3", "W:10.2.1.1", "W:10.10.1.1", "W:10.2.128.38", "W:10.10.1.43"],
    ["X:a.b.a.b", "X:a.e.a.d", "X:a.a.b.b", "X:a.b.a.c", "X:b.c.b.c", "X:a.a.a.a"],
    ["Z:asdf.exe", "Z:qwer.exe", "Z:zxcv.exe", "Z:qwer.exe", "Z:./hkjl", "Z:asdf.exe"]
):
    expected.add_edge(a, b, weight=1)
    expected.add_edge(a, c, weight=1)
    expected.add_edge(b, c, weight=1)

with suspending_memoization():
    assert_(are_graphs_equal, expected=expected, actual=build_graph_artifacts(sh, AllObjects()))

Test Building an artifact graph passed.


## Representation learning by vertex feature aggregation

The learning process is initiated with seed features, from which we compute a few iterations of ReFEx (**r**ecursive **fe**ature **ex**traction).

### Seed features

Topological seed features for a vertex $v$:

1. Vertex degree
1. Internal egonet connectivity
    - Number of edges of the subgraph $S(v)$ that are connected only between subgraph nodes distinct from $v$.
1. External egonet connectivity
    - Number of edges connected to a vertex of $S(v)$ distinct from $v$, and to a vertex outside of $S(v)$.
1. Transitivity coefficient of $S(v)$

In [43]:
import igraph as ig
import itertools as it
import numpy as np
from typing import Mapping, Dict, Tuple, Callable


Vertex2Index = Mapping[str, int]
ExtractorFeaturesInit = Callable[[ig.Graph], Tuple[np.ndarray, Vertex2Index]]


def seed_topological_basic(graph: ig.Graph) -> Tuple[np.ndarray, Vertex2Index]:
    features = np.zeros((len(graph.vs), 4))
    v2i: Dict[str, int] = {}
    for i, vertex in enumerate(graph.vs):
        family = [v.index for v in it.chain([vertex], vertex.neighbors())]
        num_edges_total = len(graph.es(_incident=family))
        num_edges_egonet = len(graph.es(_within=family))
        degree = graph.degree(vertex)
        connectivity_internal = num_edges_egonet - degree
        connectivity_external = num_edges_total - num_edges_egonet
        transitivity = graph.transitivity_local_undirected(vertices=vertex)
        
        v2i[vertex["name"]] = i
        features[i, :] = np.array([degree, connectivity_internal, connectivity_external, transitivity])
        
    return features, v2i

Graph under consideration for test:

```
(a)----(b)----(f)----(g)
 | \  / | \    |
 |  \/  |  \   |
 |  /\  |   \  |
 | /  \ |    \ |
(c)----(d)    (e)
```

In [44]:
def graph_for_refex():
    import igraph as ig
    g = ig.Graph()
    for v in "abcdefg":
        g.add_vertex(v)
    g.add_edge("a", "b")
    g.add_edge("a", "c")
    g.add_edge("a", "d")
    g.add_edge("b", "c")
    g.add_edge("b", "d")
    g.add_edge("c", "d")
    g.add_edge("b", "e")
    g.add_edge("b", "f")
    g.add_edge("e", "f")
    g.add_edge("f", "g")
    return g

In [45]:
def arrays_equal(**kwargs: np.ndarray) -> ExplanationOnFailure:
    left, right = kwargs.values()
    if left.size != right.size:
        return Explanation("Arrays of distinct size", join_args([], kwargs))
    if not np.isclose(left, right, equal_nan=True, atol=1e-5).all():
        return Explanation("Arrays not equal within tolerance", join_args([], kwargs))
    return True

In [46]:
%%test Seed feature extraction
import numpy as np

g = graph_for_refex()
features, v2i = seed_topological_basic(g)
assert_(eq, expected=dict(zip("abcdefg", range(7))), v2i=v2i)
assert_(
    arrays_equal,
    expected=np.array(
        [
            [3, 3, 2, 1.0],
            [5, 4, 1, 0.4],
            [3, 3, 2, 1.0],
            [3, 3, 2, 1.0],
            [2, 1, 4, 1.0],
            [3, 1, 3, 1.0 / 3.0],
            [1, 0, 2, np.nan]
        ],
        dtype=np.float64
    ),
    features=features
)

Test Seed feature extraction passed.


### Feature aggregation across graph topology

Necessary elements for a ReFEx-type aggregator:

1. Selection of features from the neighbours of a given vertex.
1. Configurable aggregator functions.
1. Recursive application of aggregation of selected features over all nodes.

#### Feature subset selection based on vertex neighborhood

In [47]:
import igraph as ig
from typing import List


def list_neighbor_indices(graph: ig.Graph, vertex: ig.Vertex) -> List[int]:
    return sorted([v.index for v in vertex.neighbors()])

In [48]:
@suite.test(name="List neighbor features")
def test():
    graph = graph_for_refex()
    _, vindex = seed_topological_basic(graph)
    for name, neighbours in [
        ("a", "bcd"),
        ("b", "acdfe"),
        ("c", "abd"),
        ("d", "abc"),
        ("e", "bf"),
        ("f", "beg"),
        ("g", "f")
    ]:
        nn = sorted([vindex[n] for n in neighbours])
        v, *_ = graph.vs(name=name)
        assert_(eq, neighbours=list_neighbor_indices(graph, v), expected=nn)

Test List neighbor features passed.


#### Aggregator functions

In [49]:
Aggregator = Callable[[np.ndarray], np.ndarray]  # Dimensions: m x n -> m x (2n)

In [50]:
import numpy as np


def agg_numpy(fn: Callable[..., np.ndarray]) -> Aggregator:
    def _agg(ar: np.ndarray) -> np.ndarray:
        result_masked = fn(np.ma.array(ar, mask=np.isnan(ar)), axis=0)
        result = result_masked.data
        if result_masked.mask.any():
            result[result_masked.mask] = np.nan
        return result
    return _agg


agg_sum = agg_numpy(np.sum)
agg_mean = agg_numpy(np.mean)

In [51]:
def test_arrays() -> Sequence[np.ndarray]:
    return [
        np.empty((0, 1)),
        np.empty((0, 11)),
        np.array([[3], [0], [-2]]),
        np.array([[3, 5, 0], [1, 3, -2], [-6, 8, 9], [-9, -4, 1]]),
        np.array([[4, 2], [np.nan, 12], [2, np.nan]])
    ]

In [52]:
%%test Sum aggregator
ar = test_arrays()
assert_(arrays_equal, expected=np.full((1, 1), np.nan), agg_sum=agg_sum(ar[0]))
assert_(arrays_equal, expected=np.full((1, 11), np.nan), agg_sum=agg_sum(ar[1]))
assert_(arrays_equal, expected=np.array([[-11, 12, 8]]), agg_sum=agg_sum(ar[3]))
assert_(arrays_equal, expected=np.array([[6, 14]]), agg_sum=agg_sum(ar[4]))

Test Sum aggregator passed.


In [53]:
%%test Mean aggregator
ar = test_arrays()
assert_(arrays_equal, expected=np.full((1, 1), np.nan), agg_mean1=agg_mean(ar[0]))
assert_(arrays_equal, expected=np.full((1, 11), np.nan), agg_mean2=agg_mean(ar[1]))
assert_(arrays_equal, expected=np.array([[1 / 3]]), agg_mean3=agg_mean(ar[2]))
assert_(arrays_equal, expected=np.array([[-11 / 4, 3, 2]]), agg_mean4=agg_mean(ar[3]))
assert_(arrays_equal, expected=np.array([[3, 7]]), agg_mean4=agg_mean(ar[4]))

Test Mean aggregator passed.


#### A single aggregation iteration

In [54]:
import igraph as ig
import numpy as np


def aggregate_once(graph: ig.Graph, features: np.ndarray, cols_input: int, aggregators: Sequence[Aggregator]) -> int:
    for i, agg in enumerate(aggregators, start=1):
        index_col = i * cols_input
        for v in graph.vs:
            i_neighbors = list_neighbor_indices(graph, v)
            features[v.index, index_col:index_col + cols_input] = agg(features[i_neighbors, 0:cols_input])
    return cols_input * (len(aggregators) + 1)

In [55]:
%%test Single aggregation iteration, one aggregator
G = graph_for_refex()
features = np.concatenate((
    np.array([
        [3, 3, 2, 1.0],
        [5, 4, 1, 0.4],
        [3, 3, 2, 1.0],
        [3, 3, 2, 1.0],
        [2, 1, 4, 1.0],
        [3, 1, 3, 1.0 / 3.0],
        [1, 0, 2, np.nan]
    ]),
    np.zeros((7, 4))
), axis=1)
assert_(eq, expected=8, actual=aggregate_once(G, features, 4, [agg_sum]))
assert_(
    arrays_equal,
    expected=np.array([
        [3, 3, 2, 1.0, 11.0, 10.0, 5.0, 2.4],
        [5, 4, 1, 0.4, 14.0, 11.0, 13.0, 4.0 + 1.0 / 3.0],
        [3, 3, 2, 1.0, 11.0, 10.0, 5.0, 2.4],
        [3, 3, 2, 1.0, 11.0, 10.0, 5.0, 2.4],
        [2, 1, 4, 1.0, 8.0, 5.0, 4.0, 0.4 + 1.0 / 3.0],
        [3, 1, 3, 1.0 / 3.0, 8.0, 5.0, 7.0, 1.4],
        [1, 0, 2, np.nan, 3.0, 1.0, 3.0, 1.0 / 3.0]
    ]),
    actual=features
)

Test Single aggregation iteration, one aggregator passed.


In [56]:
%%test Single aggregation iteration, two aggregators
G = graph_for_refex()
features = np.concatenate((
    np.array([
        [3, 3],
        [5, 4],
        [3, 3],
        [3, 3],
        [2, 1],
        [3, 1],
        [1, 0]
    ]),
    np.zeros((7, 4))
), axis=1)
assert_(eq, expected=6, actual=aggregate_once(G, features, 2, [agg_sum, agg_mean]))
assert_(
    arrays_equal,
    expected=np.array([
        [3, 3, 11.0, 10.0, 11.0 / 3.0, 10.0 / 3.0],
        [5, 4, 14.0, 11.0, 14.0 / 5.0, 11.0 / 5.0],
        [3, 3, 11.0, 10.0, 11.0 / 3.0, 10.0 / 3.0],
        [3, 3, 11.0, 10.0, 11.0 / 3.0, 10.0 / 3.0],
        [2, 1, 8.0, 5.0, 4.0, 2.5],
        [3, 1, 8.0, 5.0, 8.0 / 3.0, 5.0 / 3.0],
        [1, 0, 3.0, 1.0, 3.0, 1.0]
    ]),
    actual=features
)

Test Single aggregation iteration, two aggregators passed.


#### Bringing it all together

In [57]:
import igraph as ig
import numpy as np


def aggregate_features(
    graph: ig.Graph,
    seed: np.ndarray,
    aggregators: Sequence[Aggregator],
    num_iter: int = 3
) -> np.ndarray:
    num_vertices, num_seeds = seed.shape

    def num_features_after_iter(i: int) -> int:
        return num_seeds * (len(aggregators) + 1) ** i

    features = np.zeros((num_vertices, num_features_after_iter(num_iter)))
    features[:, 0:num_seeds] = seed
    for i in range(num_iter):
        aggregate_once(graph, features, num_features_after_iter(i), aggregators)
    return features

In [58]:
%%test Aggregation: two features, one aggregator, one iteration
G = graph_for_refex()
seed = np.array([
    [3, 3, 2, 1.0],
    [5, 4, 1, np.nan],
    [3, 3, 2, 1.0],
    [3, 3, 2, 1.0],
    [2, 1, 4, 1.0],
    [3, 1, 3, 0.4],
    [1, 0, 2, np.nan]
])
assert_(
    arrays_equal,
    expected=np.array([
        [3, 3, 11.0, 10.0],
        [5, 4, 14.0, 11.0],
        [3, 3, 11.0, 10.0],
        [3, 3, 11.0, 10.0],
        [2, 1, 8.0, 5.0],
        [3, 1, 8.0, 5.0],
        [1, 0, 3.0, 1.0]
    ]),
    actual=aggregate_features(G, seed, [agg_sum], 1)
)

Issue ** Test Aggregation: two features, one aggregator, one iteration ** Failure
Arrays of distinct size:

    actual   => [[ 3.   3.   2.   1.  11.  10.   5.   2. ]
 [ 5.   4.   1.   nan 14.  11.  13.   4.4]
 [ 3.   3.   2.   1.  11.  10.   5.   2. ]
 [ 3.   3.   2.   1.  11.  10.   5.   2. ]
 [ 2.   1.   4.   1.   8.   5.   4.   0.4]
 [ 3.   1.   3.   0.4  8.   5.   7.   1. ]
 [ 1.   0.   2.   nan  3.   1.   3.   0.4]]
    expected => [[ 3.  3. 11. 10.]
 [ 5.  4. 14. 11.]
 [ 3.  3. 11. 10.]
 [ 3.  3. 11. 10.]
 [ 2.  1.  8.  5.]
 [ 3.  1.  8.  5.]
 [ 1.  0.  3.  1.]]


In [59]:
%%test Aggregation: 3 features, 2 aggregators, 2 iterations
G = graph_for_refex()
seed = np.array([
    [3, 3, 1.0],
    [5, 4, 0.4],
    [3, 3, 1.0],
    [3, np.nan, 1.0],
    [2, 1, 1.0],
    [3, 1, 1.0 / 3.0],
    [1, 0, np.nan]
])
assert_(
    arrays_equal,
    expected=np.array([
        [3, 3     , 1       , 11,  7, 2.4     , 3.666666, 3.5     , 0.8     , 11,  7, 2.4     , 36, 25, 9.133333, 10.133333,  8.833333, 2.466666, 3.666666, 3.5     , 0.8     , 12       , 8.333333, 3.044444, 3.377777, 2.944444, 0.822222],
        [5, 4     , 0.4     , 14,  8, 4.333333, 2.8     , 2       , 0.866666, 14,  8, 4.333333, 49, 34, 9.333333, 17.666666, 14.5     , 3.466666, 2.8     , 2       , 0.866666,  9.8     , 6.8     , 1.866666, 3.533333, 2.9     , 0.693333],
        [3, 3     , 1.0     , 11,  7, 2.4     , 3.666666, 3.5     , 0.8     , 11,  7, 2.4     , 36, 25, 9.133333, 10.133333,  8.833333, 2.466666, 3.666666, 3.5     , 0.8     , 12       , 8.333333, 3.044444, 3.377777, 2.944444, 0.822222],
        [3, np.nan, 1.0     , 11, 10, 2.4     , 3.666666, 3.333333, 0.8     , 11, 10, 2.4     , 36, 22, 9.133333, 10.133333,  9       , 2.466666, 3.666666, 3.333333, 0.8     , 12       , 7.333333, 3.044444, 3.377777, 3       , 0.822222],
        [2, 1     , 1.0     ,  8,  5, 0.733333, 4       , 2.5     , 0.366666,  8,  5, 0.733333, 22, 13, 5.733333,  5.466666,  3.666666, 1.566666, 4       , 2.5     , 0.366666, 11       , 6.5     , 2.866666, 2.733333, 1.833333, 0.783333],
        [3, 1     , 0.333333,  8,  5, 1.4     , 2.666666, 1.666666, 0.7     ,  8,  5, 1.4     , 25, 14, 5.4     ,  9.8     ,  5.5     , 1.566666, 2.666666, 1.666666, 0.7     ,  8.333333, 4.666666, 1.8     , 3.266666, 1.833333, 0.522222],
        [1, 0     , np.nan  ,  3,  1, 0.333333, 3       , 1       , 0.333333,  3,  1, 0.333333,  8,  5, 1.4     ,  2.666666,  1.666666, 0.7     , 3,        1       , 0.333333,  8       , 5       , 1.4     , 2.666666, 1.666666, 0.7]
    ]),
    actual=aggregate_features(G, seed, [agg_sum, agg_mean], 2)
)

Test Aggregation: 3 features, 2 aggregators, 2 iterations passed.


### Dimension reduction



The dimension reduction approach taken by [Palladino and Thissen](https://arxiv.org/abs/1812.02848) consists of a nonnegative factorization of the feature matrix obtained by aggregation. The reduced features are even normalized so they may be interpreted as *role belonging probabilities*. Since this role belonging property caters to the authors' approach to anomaly detection out of the vector signals, we will compute this normalization once we reach this later stage.

#### Non-negative matrix factorization (classic RolX)

In [60]:
import numpy as np
import sklearn.decomposition as skd


class MethodFailedToConverge(Exception):
    pass


def reduce_dim_nmf(
    features: np.ndarray,
    dim: int,
    max_iter: int = 500,
    tol: float = 1e-4,
    raise_on_convergence_failure: bool = True
) -> np.ndarray:
    _, num_features = features.shape
    if dim >= num_features:
        raise ValueError(
            f"Provided dimension {dim} is not a reduction from current vertex embedding dimension {num_features}"
        )
    model = skd.NMF(n_components=dim, solver="mu", beta_loss="kullback-leibler", tol=tol, max_iter=max_iter)
    features_reduced = model.fit_transform(features)
    if raise_on_convergence_failure and model.n_iter_ >= max_iter:
        raise MethodFailedToConverge()
    return features_reduced

In [61]:
%%test Reducing dimension of a matrix
R = reduce_dim_nmf(np.random.rand(100, 100), 10, max_iter=500)
assert_(eq, Rshape=R.shape, expected=(100, 10))
assert (R >= 0.0).all()

Test Reducing dimension of a matrix passed.


In [62]:
%%test Raising on failure to converge
try:
    R = reduce_dim_nmf(np.random.rand(100, 100), 10, max_iter=10)
    fail("Supposed to raise because the factorization would have failed to converge.")
except MethodFailedToConverge:
    pass

Test Raising on failure to converge passed.


/home/ben/miniconda3/envs/crab/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1075: ConvergenceWarning: Maximum number of iteration 10 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iteration %d reached. Increase it to"


# Test result summary

In [63]:
if __name__ == "__main__":
    _ = summarize_results(suite)

32 passed, 0 failed, 0 raised an error
